In [512]:
import os
import time
import re
import pandas as pd
from tqdm import tqdm
import multiprocessing
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options

In [526]:
# Path to the Edge WebDriver executable
edge_driver_path = r"C:\msedgedriver.exe"

edge_options = Options()
edge_options.add_argument('--headless')
edge_options.add_argument('--no-sandbox')
edge_options.add_argument('--disable-dev-shm-usage')

service = Service(edge_driver_path)

# Create a new instance of the Edge driver
driver = webdriver.Edge(service=service, options=edge_options)

In [532]:
def scrape_map_ids(url_list):

    pattern = r'/maps/([a-f0-9]{24})'
    map_id_list = []
    
    for url in tqdm(url_list):
        driver.get(url)
        time.sleep(5)
        html_content = driver.page_source
        soup = BeautifulSoup(html_content, 'html.parser')
        matches = re.findall(pattern, soup.prettify())
        map_urls = ["https://www.geoguessr.com/maps/" + m for m in matches]
        for map in map_urls:
            if map not in url_list:
                map_id_list.append(map)
    
    return list(set(map_id_list))

In [ ]:
map_id_list = scrape_map_ids(url_list)

In [ ]:
folder_path = 'all_html_files'

# Create the folder if it doesn't exist
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

for url in tqdm(map_id_list):
    driver.get(url)
    time.sleep(5)
    html_content = driver.page_source
    file_name = url.split("/")[-1] + '.html'
    file_path = os.path.join(folder_path, file_name)
    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(html_content)